In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Working with PDFs as images

### Approach 2: Use the entire page!

How do we mentally split PDFs? We usually think of them as a set of pages. We can do the same with PDFs, by embedding the entire page!

<img src="data/imgs/hai_ai-index-report-2025_chapter2_final_07_of_86.jpg" width="200px" />
<img src="data/imgs/hai_ai-index-report-2025_chapter2_final_32_of_86.jpg" width="200px" />
<img src="data/imgs/hai_ai-index-report-2025_chapter2_final_49_of_86.jpg" width="200px" />

In [2]:
import weaviate
import os

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=os.getenv("WEAVIATE_API_KEY"),
    headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
        "X-Anthropic-Api-Key": os.getenv("ANTHROPIC_API_KEY"),
    },
)

/Users/jphwang/code/demo-code/202507-lipari/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/jphwang/code/demo-code/202507-lipari/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/jphwang/code/demo-code/202507-lipari/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update the gencode to avo

In [3]:
client.collections.delete("Pages")

In [4]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Pages",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="page_image",
            data_type=DataType.BLOB,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        # Add `Configure.Vectors.multi2vec_cohere` vector to the collection with:
        # name: "default", source properties: ["page_image"], and model: "embed-v4.0"
        # BEGIN_SOLUTION
        Configure.Vectors.multi2vec_cohere(
            name="default",
            image_fields=["page_image"],
            model="embed-v4.0"
        )
        # END_SOLUTION
    ]
)

In [5]:
pages = client.collections.get("Pages")

Load pre-computed vectors & metadata

In [6]:
import numpy as np
import json

with open("data/embeddings/embeddings_metadata.json", "r") as f:
    metadata = json.load(f)

embeddings = np.load("data/embeddings/image_embeddings.npy")

Import data

In [7]:
from tqdm import tqdm
from pathlib import Path
import base64

with pages.batch.fixed_size(batch_size=100) as batch:
    for i, embedding in tqdm(enumerate(embeddings)):
        filepath = Path(metadata["image_paths"][i])
        image = filepath.read_bytes()
        base64_image = base64.b64encode(image).decode('utf-8')
        obj = {
            "document_title": "Bosch WGG254Z0GR Manual",
            "page_image": base64_image,
            "filename": filepath.name
        }

        # Add object to batch for import with (batch.add_object())
        # This time, manually provide the vector with `{"default": embedding}`
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj,
            vector={"default": embedding}
        )
        # END_SOLUTION

56it [00:00, 2332.55it/s]


In [8]:
from weaviate.classes.generate import GenerativeConfig, GenerativeParameters

prompt = GenerativeParameters.grouped_task(
    prompt="How do I clean the drain pump? Please describe the steps in detail based on the provided images.",
    image_properties=["page_image"],  # Property containing images in Weaviate
)


response = pages.generate.near_text(
    # Try a RAG query with:
    # query (what to search for): "How to clean the drain pump" and
    # limit (how many objects to fetch): 3
    # grouped_task (prompt): prompt defined above
    # BEGIN_SOLUTION
    query="How to clean the drain pump",
    limit=3,
    grouped_task=prompt,
    # END_SOLUTION
    # Runtime definition of what generative AI model provider to use
    generative_provider=GenerativeConfig.anthropic()
)

In [9]:
print(response.generative.text)

Based on the images provided, here are the detailed steps to clean the drain pump:

1. Ensure the drain pump is empty. Refer to page 36 for emptdrain pump.

2. Since water may remain in the drain pump, unscrew the pump cap carefully.
   - The filter insert in the pump housing may become stuck due to coarse particles of dirt. Loosen the dirt and remove the filter insert.

3. Clean the interior, the thread on the pump cap, and the pump housing.
   - The pump cap is comprised of two parts which can be taken apart for cleaning.

4. Ensure that the impeller in the drain pump can rotate.

5. Insert the pump cap.
   - Ensure that the parts of the pump cap are correctly assembled.

6. Screw the pump cap as far as it will go.
   - The handle on the pump cap must be in a vertical position.

7. Insert and close the service flap.

It's recommended to clean the drain pump regularly, at least once a year, as well as in the event of faults like blockages or rattling noises.

Before the next wash:
1. 

In [10]:
for o in response.objects:
    print(f"Filename: {o.properties['filename']}")

Filename: manual_bosch_WGG254Z0GR_38_of_56.jpg
Filename: manual_bosch_WGG254Z0GR_36_of_56.jpg
Filename: manual_bosch_WGG254Z0GR_39_of_56.jpg


In [11]:
client.close()